In [24]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, Trainer, EvalPrediction
import os
#%pip install scipy
import scipy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [25]:
#data import 
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
#display (train_df)
print (os.listdir('data'))
dataset = Dataset.from_pandas(train_df)
#data_files = {"train": "train.csv", "test": "test.csv"}
#dataset = load_dataset("data", data_files=data_files)
print (dataset)

['test.csv', 'train.csv']
Dataset({
    features: ['qa_id', 'question_title', 'question_body', 'question_user_name', 'question_user_page', 'answer', 'answer_user_name', 'answer_user_page', 'url', 'category', 'host', 'question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 

In [26]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, DefaultDataCollator, XLNetConfig, XLNetLMHeadModel, XLNetModel
model_name = 'xlnet-base-cased' # 'xlnet-large-cased', 'tiny-xlnet-base-cased', 'jkgrad/xlnet-base-squadv2'

data_collator = DefaultDataCollator()
tokenizer = XLNetTokenizer.from_pretrained(model_name)
config = XLNetConfig.from_pretrained(
    "roberta-base",
    num_labels=1,
    id2label={ 0: "👎", 1: "👍"},
)
model = XLNetLMHeadModel.from_pretrained(
    "roberta-base",
    config=config,
)
#model = XLNetForSequenceClassification.from_pretrained(model_name)

loading file https://huggingface.co/xlnet-base-cased/resolve/main/spiece.model from cache at C:\Users\km201/.cache\huggingface\transformers\df73bc9f8d13bf2ea4dab95624895e45a550a0f0a825e41fc25440bf367ee3c8.d93497120e3a865e2970f26abdf7bf375896f97fde8b874b70909592a6c785c9
loading file https://huggingface.co/xlnet-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/xlnet-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/xlnet-base-cased/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at C:\Users\km201/.cache\huggingface\transformers\06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type"

In [27]:

#model.add_adapter("covid_qa") #, config=adapter_config)
#model.train_adapter("covid_qa")
#tokenized_covid_dataset = covid_dataset.map(preprocess_function, batched=True, remove_columns=covid_dataset["train"].column_names)

In [28]:

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6079
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4560
  0%|          | 0/4560 [00:00<?, ?it/s]

ValueError: You have to specify either input_ids or inputs_embeds

In [ ]:
class AvgPooledXLNet(XLNetModel):
    def forward(self, ids, seg_ids=None):
        att_mask = (ids > 0).float()
        x_bert = super().forward(ids, att_mask, token_type_ids=seg_ids)[0]
        att_mask = att_mask.unsqueeze(-1)
        return (x_bert * att_mask).sum(dim=1) / att_mask.sum(dim=1)
class CustomXLNet(nn.Module):
    def __init__(self, n_h, n_feats, head_dropout=0.2):
        super().__init__()
        config = XLNetConfig.from_json_file(PRETRAINED_PATH + 'xlnet-base-cased/config.json')
        self.xlnet = AvgPooledXLNet(config)
        self.head = Head2(n_h, n_feats, n_bert=768, dropout=head_dropout)

    def forward(self, x_feats, q_ids, a_ids, seg_q_ids=None, seg_a_ids=None):
        x_q_bert = self.xlnet(q_ids, seg_q_ids)
        x_a_bert = self.xlnet(a_ids, seg_a_ids)
        return self.head(x_feats, x_q_bert, x_a_bert)
def get_preds(train, test, ModelClass, tokenizer, model_name, checkpoint_dir, folds):

    seg_ids_test, ids_test = {}, {}
    max_seq_len = 512
    for mode, df in [('test', test)]:
        for text, cols in [('question', ['question_title', 'question_body']), 
                            ('answer', ['question_title', 'answer'])]:
            ids, seg_ids = [], []
            for x1, x2 in tqdm(df[cols].values):
                encoded_inputs = tokenizer.encode_plus(
                    x1, x2, add_special_tokens=True, max_length=max_seq_len, pad_to_max_length=True, 
                    return_token_type_ids=True
                )
                ids.append(encoded_inputs['input_ids'])
                seg_ids.append(encoded_inputs['token_type_ids'])
            ids_test[text] = np.array(ids)
            seg_ids_test[text] = np.array(seg_ids)

    train_category, test_category, category_dict, category_dict_reverse = \
        get_categorical_features(train, test, 'category')

    cat_features_train = train_category.reshape(-1, 1)
    cat_features_test = test_category.reshape(-1, 1)
    ohe = OneHotEncoder(handle_unknown='ignore')
    ohe.fit(cat_features_train)
    cat_features_test = ohe.transform(cat_features_test).toarray()

    num_workers = 8
    device = 'cuda'

    bs_test = 2
    test_loader = DataLoader(
        TextDataset5(cat_features_test, ids_test['question'], ids_test['answer'], 
                        seg_ids_test['question'], seg_ids_test['answer'], test.index),
        batch_size=bs_test, shuffle=False, num_workers=num_workers
    )

    init_seed()
    preds = np.zeros((len(test), N_TARGETS))
    for fold_id in folds:
        checkpoint_file = f'{checkpoint_dir}{model_name}_fold_{fold_id + 1}_best.pth'
        model = ModelClass(256, cat_features_test.shape[1]).to(device)
        test_preds = infer(model, test_loader, checkpoint_file, device)
        preds += test_preds / len(folds)

    return preds

def get_xlnet_preds(train, test):
    tokenizer = XLNetTokenizer.from_pretrained(PRETRAINED_PATH + 'xlnet-base-cased/')
    model_name = 'siamese_xlnet_1_comb'
    checkpoint_dir = '../input/siamese-xlnet-models-1/'
    return get_preds(train, test, CustomXLNet, tokenizer, model_name, checkpoint_dir, [0, 1, 2, 4, 5, 7, 8])